# edgedbb
testing the edgedb database from rust

In [2]:
:dep thiserror = "1.0.49"
:dep serde = "1.0.189"
:dep serde_json = "1.0.107"
:dep tokio = { version = "1.33.0", features = ["full"] }
:dep edgedb-tokio = "0.5.0"
:dep edgedb-protocol = "0.6.0"
:dep edgedb-derive = "0.5.1"

## Edgedb Cli

#### setup

create a new project in the root directory of the code repository by running the following commands. Note that edgedb shoul already be running in a container using docker, kubernetes or any other containerised tool.

```sh
# list all edgedbb instances
❯ edgedb instance list
┌────────┬───────────┬─────────────────┬──────────────┬──────────┐
│ Kind   │ Name      │ Location        │ Version      │ Status   │
├────────┼───────────┼─────────────────┼──────────────┼──────────┤
│ local  │ labs      │ localhost:10705 │ 2.12+f3129ef │ inactive │
│ local  │ tenchi    │ localhost:10706 │ 2.12+f3129ef │ inactive │
│ remote │ localhost │ localhost:5656  │ 3.4+7598b99  │ up       │
└────────┴───────────┴─────────────────┴──────────────┴──────────┘
```

instantiate a new project (choose remote instance name e.g. localhost)
```sh
# instantiate a new project
❯ edgedb project init
```

check that the project is connected to the instance

```sh
❯ edgedb project info
┌───────────────┬────────────────────────────────────────────┐
│ Instance name │ localhost                                  │
│ Project root  │ /Users/goose/Spaces/pepeye/github/ai/mlops │
└───────────────┴────────────────────────────────────────────┘
```

#### migrations

```sh
# restore database
❯ edgedb restore -I localhost path/to/dump/file/edgedb.dump
```

import package dependencies

In [3]:
use edgedb_tokio::create_client;
use edgedb_tokio::Client;
use edgedb_derive::Queryable;
use serde::{Serialize, Deserialize};

In [7]:
// connect to database
let client = create_client()
    .await
    .expect("Client should have initiated");

// query
let val: i64 = client
    .query_required_single("select 7*8", &())
    .await
    .expect("Query should have worked");

// print resulting value
println!("[out]: 7*8 is: {val}");

[out]: 7*8 is: 56


In [8]:
// Without arguments: just add &() after the query
let query_a: String =
    client.query_required_single("select 'Just a string'", &()).await?;

// With arguments, same output as the previous example
let a = " a ";
let b = "string";
let query_b: String = client
    .query_required_single("select 'Just' ++ <str>$0 ++ <str>$1", &(a, b))
    .await?;

// print resulting value
println!("[out]: {}", query_a);
println!("[out]: {}", query_b);

[out]: Just a string
[out]: Just a string


### Test Data
The following data will be used to create a record in edgedb and used for retrieval purposes
```json
{
    "firstname": "tony",
    "lastname": "stark",
    "nickname": "ironman",
    "status": "PENDING",
    "email": "ironman@avengers.com",
    "login": "@ironman",
    "password": "$2b$12$lH23QLU6pRf9g8jJW91HvOrPbGDFzwZ6x8Lz0jKkMQ7Bmgf1Sw9He",
    "preferred": true,
    "provider": "ardency"
  }
```

In [ ]:
use edgedb_derive::Queryable;
use serde::{Serialize, Deserialize};

let query = r#"
        with data := <json>$0
        insert Person {
          firstname := <str>data['firstname'],
          lastname :=  <str>data['lastname'],
          nickname := <str>data['nickname'],
          status :=  <Status>data['status'],
          identity := (insert Identity {
            email := <str>data['email'],
            login := <str>data['login'],
            password := <str>data['password'],
            preferred := <bool>data['preferred'],
            provider := (select (
              insert Provider { name := <optional str>data['provider'] }
              unless conflict on .name 
              else 
                (select Provider filter .name = <optional str>data['provider'] Limit 1)
            ))
          })
        }
        "

let data = serde_json::to_string(&person)?;
let id = self.pool.query(query, &(data,)).await?;
Ok(id)

In [22]:
use edgedb_derive::Queryable;
use serde::{Serialize, Deserialize};

#[derive(Debug, Serialize, Deserialize, Queryable)]
pub struct Hero {
    name: String,
    email: String,
    identity: Identity,
    powers: Vec<String>,
}

#[derive(Debug, Serialize, Deserialize, Queryable)]
pub struct Identity {
    firstname: String,
    lastname: String,
}

// make pool connection to database
// let pool = client.create_client().await?;

// define a query
let query = r#"
    select Hero {
      name,
      email,
      identity: {
        firstname,
        lastname
      },
      powers
    }
    filter .name = <str>$0
    limit 1
"#;

// // fetch data
let name = "spiderman";
let r: Option<Hero> = client.query_single(query, &(name,)).await?;
println!("{:#?}", r);

Some(
    Hero {
        name: "spiderman",
        email: "spidey@avengers.com",
        identity: Identity {
            firstname: "peter",
            lastname: "parker",
        },
        powers: [
            "strength",
            "speed",
            "reflexes",
            "healing",
            "spider-sense",
        ],
    },
)
